In [34]:
import pandas as pd
import os
import json
import glob
from shapely.geometry  import shape, Point
from geopy.geocoders import Nominatim

In [35]:
geolocator = Nominatim(user_agent="test_tk")

In [36]:
os.makedirs('./results', exist_ok=True )

In [37]:
data_path = '../download_with_python/data/'
cur_dir = os.getcwd()

In [38]:
os.chdir(data_path)
files = glob.glob("*.json")
os.chdir(cur_dir)

In [39]:
crimes_dict = {}
offenders_dict = {}
env_dict = {}
i = 0

for file in files:
    name = file.split(sep="_")[0]
    if name == 'crime':
        crimes_dict[name+f'_{i}'] = pd.read_json((data_path+file), lines=True)
    elif name == 'offenders':
        offenders_dict[name+f'_{i}'] = pd.read_json((data_path+file), lines=True)
        # drop registered as homeless offenders
        offenders_dict[name+f'_{i}'] = offenders_dict[name+f'_{i}'][~(offenders_dict[name+f'_{i}']['block'] == '(registered as homeless)')]
        # replace anonymized house number with 0 (replaces X with 0)
        offenders_dict[name+f'_{i}']['block'] = offenders_dict[name+f'_{i}']['block'].str.replace('X','0') + ' CHICAGO'
    elif name == 'envcomplaint':
        env_dict[name+f'_{i}'] = pd.read_json((data_path+file), lines=True)
    else:
        print(f'error on {file}')


In [40]:
# TO DO
# make merger function to merge like for like

In [41]:
# define geojsons
with open('../../geojsons/Boundaries - Community Areas (current).geojson') as f:
    commarea = json.load(f)

In [42]:
def merge_df(dictionary):
    merged_df = pd.DataFrame()
    for datfram in dictionary:
        merged_df= pd.concat([merged_df,dictionary[datfram]],axis=0)
    return merged_df

In [43]:
merged_crimes = merge_df(crimes_dict)
merged_offenders = merge_df(offenders_dict)
merged_envcomplaints = merge_df(env_dict)

In [50]:
def get_commarea_env(dataframe):
    dataframe['community_area']=''
    for row in dataframe.index:
        point = Point(dataframe['location.coordinates'][row][0],dataframe['location.coordinates'][row][1])
        for feature in commarea['features']:
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                dataframe['community_area'][row] = feature['properties']['community']

In [68]:
def get_commarea_offenders(dataframe):
    dataframe['community_area']=''
    i=0
    for row in dataframe.index:
        try:
            location = geolocator.geocode(dataframe['block'][row])
            latlong = [location.latitude, location.longitude]
            point = Point(latlong[1], latlong[0])
            for feature in commarea['features']:
                polygon = shape(feature['geometry'])
                if polygon.contains(point):
                    dataframe['community_area'][row] = feature['properties']['area_numbe']
                else:
                    dataframe['community_area'][row] = 'no match to community_area'
        except:
            dataframe['community_area'][row] = 'no match to osm'
        print(f'{round((i/dataframe.shape[0]),4)*100}% done')
        i+=1


In [83]:
location = geolocator.geocode(merged_offenders['block'][10])
latlong = [location.latitude, location.longitude]
point = Point(latlong[1], latlong[0])
for feature in commarea['features']:
    polygon = shape(feature['geometry'])
if polygon.contains(point):
    merged_offenders['community_area'][5] = feature['properties']['area_numbe']
else:
    merged_offenders['community_area'][5] = 'no match to community_area'

AttributeError: 'NoneType' object has no attribute 'latitude'

In [70]:
get_commarea_env(merged_envcomplaints)
get_commarea_offenders(merged_offenders)

C:\Users\tkkim\AppData\Local\Temp\ipykernel_12700\1914138407.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['community_area'][row] = feature['properties']['community']
C:\Users\tkkim\AppData\Local\Temp\ipykernel_12700\3323343338.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['community_area'][row] = 'no match to community_area'
C:\Users\tkkim\AppData\Local\Temp\ipykernel_12700\3323343338.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

0.0% done
0.03% done
0.06999999999999999% done
0.1% done


C:\Users\tkkim\AppData\Local\Temp\ipykernel_12700\3323343338.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['community_area'][row] = 'no match to osm'


0.13999999999999999% done
0.16999999999999998% done
0.21% done
0.24% done
0.27999999999999997% done
0.31% done
0.35000000000000003% done
0.38% done
0.42% done
0.44999999999999996% done
0.49% done
0.52% done
0.5599999999999999% done
0.59% done
0.63% done
0.66% done
0.7000000000000001% done
0.73% done
0.77% done
0.8% done
0.84% done
0.8699999999999999% done
0.91% done
0.9400000000000001% done
0.97% done
1.01% done
1.04% done
1.08% done
1.11% done
1.15% done
1.18% done
1.22% done
1.25% done
1.29% done
1.32% done
1.3599999999999999% done
1.39% done
1.43% done
1.46% done
1.5% done
1.53% done
1.5699999999999998% done
1.6% done
1.6400000000000001% done
1.67% done
1.71% done
1.7399999999999998% done
1.78% done
1.81% done
1.8499999999999999% done
1.8800000000000001% done
1.92% done
1.95% done
1.9800000000000002% done
2.02% done
2.0500000000000003% done
2.09% done
2.12% done
2.16% done
2.19% done
2.23% done
2.26% done
2.3% done
2.33% done
2.37% done
2.4% done
2.44% done
2.4699999999999998% done


In [71]:
merged_crimes.to_csv('./results/crimes_cleaned.csv', index=False)
merged_offenders.to_csv('./results/offeders_cleaned.csv', index=False)
merged_envcomplaints.to_csv('./results/crimes_envcomplaints.csv', index=False)

In [72]:
grouped_offenders_by_commarea = merged_offenders.groupby('community_area').agg(total_offenders_in_ca=('last','count')).reset_index()
grouped_offenders_by_commarea.to_csv('./results/offenders_by_community_area.csv', index=False)

In [73]:
grouped_complaints_by_commarea = merged_envcomplaints.groupby('community_area').agg(total_complaints_in_ca=('complaint_id','count')).reset_index()
grouped_complaints_by_commarea.to_csv('./results/envcomplaints_by_community_area.csv', index=False)

In [79]:
grouped_offenders_by_commarea.head()

,community_area,total_offenders_in_ca
0,9,1
1,no match to community_area,1882
2,no match to osm,989


In [76]:
merged_crimes.dtypes

id                                 int64
case_number                       object
date                      datetime64[ns]
block                             object
iucr                              object
primary_type                      object
description                       object
location_description              object
arrest                              bool
domestic                            bool
beat                               int64
district                           int64
ward                               int64
community_area                     int64
fbi_code                          object
x_coordinate                     float64
y_coordinate                     float64
year                               int64
updated_on                        object
latitude                         float64
longitude                        float64
location.latitude                float64
location.longitude               float64
location.human_address            object
dtype: object

In [75]:
big_df = merged_crimes.merge(grouped_offenders_by_commarea, how='left', on='community_area')

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat